In [1]:
import os
os.environ['RNETEB_PATH'] = '/home/users/gtully/RNET-EB'
os.environ['RANGER_PATH'] = '/home/users/gtully/Ranger-Deep-Learning-Optimizer/'

In [2]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
import random
import ast
import os
from torch.utils.data import Dataset, DataLoader
from ast import literal_eval
import sklearn
from Bio import SeqIO
from sklearn.model_selection import train_test_split
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import sys
import yaml

sys.path.append(os.environ['RNETEB_PATH']+ '/tools')
sys.path.append(os.environ['RANGER_PATH'] + '/ranger')
from plotting import *   # from RNETEB tools 
from training import *   # from RNETEB tools

from ranger import Ranger # from Ranger-Deep-Learning-Optimizer cloned

# RNET_EB_000 Training Important Notes

1) This model was trained on both the logkd_lig_scaled and the logkd_nolig_scaled, per riboswitch sequence (see the uniquely defined RNA_Dataset in this notebook). 

2) Hyperparameters were chosen from success of fine-tuning RibonanzaNet on secondary structure data. 
    - optimizer: Ranger
    - scheduler: CosineAnnealingLR
    - criterion: L1

3) Before running all cells in the NB, make sure to clone both RNET-EB and the Ranger-Deep-Learning-Optimizer Repository and add paths to : 
    os.environ['RNETEB_PATH'] = '/your/path/to/RNET-EB'
    os.environ['RANGER_PATH'] = '/your/path/to/Ranger-Deep-Learning-Optimizer/'

4) As is, this repository regenerates data in locations where I already have data saved in public RNET-EB repository. If you would like to save results separately, rename the output files. 

   

## Set random state for consistency 

In [3]:
# Set seed for reproducibility
SEED = 0

random.seed(SEED)

# NumPy
np.random.seed(SEED)

# torch 
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # for multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Import Cleaned Data from RibonanzaNet_EB_data_prep.ipynb

In [4]:
train_df = pd.read_json(os.environ['RNETEB_PATH'] + '/data/processed_data/RNET_EB_train.json')
val_df = pd.read_json(os.environ['RNETEB_PATH'] + '/data/processed_data/RNET_EB_val.json')
test_df = pd.read_json(os.environ['RNETEB_PATH'] + '/data/processed_data/RNET_EB_test.json')

In [5]:
train_df.columns

Index(['Activation Ratio', 'Design', 'Folding_Subscore', 'KDFMN', 'KDOFF',
       'KDON', 'KDnoFMN', 'Kd_OFF', 'Kd_ON', 'NumberOfClusters', 'Player',
       'Puzzle_Name', 'Round', 'index', 'ligand', 'min_kd_val', 'puzzle',
       'sequence', 'switch', 'MS2_aptamer', 'lig_aptamer', 'MS2_lig_aptamer',
       'constraints_worked', 'logkd_nolig', 'logkd_lig', 'logkd_nolig_scaled',
       'logkd_lig_scaled', 'Dataset', 'passed_CDHIT_filter', 'log_AR', 'id',
       'description'],
      dtype='object')

# Define Dataset

In [6]:
class RNA_Dataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokens = {nt: i for i, nt in enumerate('ACGU')}
        self.label_names = ['logkd_lig_scaled', 'logkd_nolig_scaled']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence=[self.tokens[nt] for nt in (self.data.loc[idx,'sequence'])]
        sequence=np.array(sequence)
        sequence=torch.tensor(sequence)

        labels = np.array([self.data.loc[idx, l] for l in self.label_names])  # Just 1 value per label
        labels = torch.tensor(labels, dtype=torch.float32)  # Ensure labels are of correct float type


        return {'sequence': sequence, 'labels': labels}



In [7]:
# Create the datasets
train_dataset = RNA_Dataset(train_df)
val_dataset = RNA_Dataset(val_df)

# Test the first sample
print(train_dataset[0]['labels'].shape)


torch.Size([2])


In [8]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

In [9]:
sys.path.append(os.environ['RNETEB_PATH']+'/ribonanzanet2d-final')

from Network import *

class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)
        self.entries=entries

    def print(self):
        print(self.entries)

def load_config_from_yaml(file_path):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
    return Config(**config)

class finetuned_RibonanzaNet(RibonanzaNet):
    def __init__(self, config, pretrained=False):
        super(finetuned_RibonanzaNet, self).__init__(config)
        if pretrained:
            self.load_state_dict(torch.load(os.environ['RNETEB_PATH']+'/ribonanzanet-weights/RibonanzaNet.pt',map_location='cpu'))

        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.decoder = nn.Linear(64, 2)  # From 64 "pooled values from each channel " to 2 output labels

    def forward(self,src):

        sequence_features, pairwise_features=self.get_embeddings(src, torch.ones_like(src).long().to(src.device))
        pairwise_features = pairwise_features.squeeze(0)  # Remove the batch dimension to make it [H, W, 64]
        pairwise_features = pairwise_features.permute(2, 0, 1)  # Change to [64, H, W] to match pooling expectation (C, H, W)

        # Apply global average pooling, result is [64, 1, 1]
        pairwise_features = self.global_pool(pairwise_features)

        # Flatten the output to [64]
        pairwise_features = pairwise_features.view(pairwise_features.size(0))  # Flatten to [64] (batch size 1, so this will be [64])

        # Pass through the decoder to get the final output [2]
        output = self.decoder(pairwise_features)

        return output

In [10]:
config=load_config_from_yaml(os.environ['RNETEB_PATH'] + "/ribonanzanet2d-final/configs/pairwise.yaml")
model=finetuned_RibonanzaNet(config,pretrained=True).cuda()



constructing 9 ConvTransformerEncoderLayers


## Training Loop

Note: Need to clone the Ranger-Deep-Learning-Optimizer and add the path to the ranger folder within the repository. 

### Save the epoch losses as well as weights and model check point. 
### Also save plots of training and validation losses. 

In [11]:
# Initialize Hyperparameters
epochs = 20
cos_epoch = 15

best_loss = np.inf
optimizer = Ranger(model.parameters(), weight_decay=0.001, lr=0.0001)
criterion = torch.nn.L1Loss()
schedule = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(epochs - cos_epoch) * len(train_loader))

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [ ]:

# Setup directories
checkpoint_dir = os.path.join(os.environ['RNETEB_PATH'], 'results/checkpoints/RNETEB_000')
figure_dir = os.path.join(os.environ['RNETEB_PATH'], 'results/figures/RNETEB_000/training_curves')
Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
Path(figure_dir).mkdir(parents=True, exist_ok=True)

# Track losses
train_losses = []
val_losses = []

for epoch in range(epochs):
    model.train()
    tbar = tqdm(train_loader)
    total_loss = 0
    oom = 0
    for idx, batch in enumerate(tbar):
        sequence = batch['sequence'].cuda()
        labels = batch['labels'].cuda()
        output = model(sequence) 
        labels = labels.view_as(output)
        
        # Compute loss
        loss = criterion(output, labels)
        loss = loss.mean()

        # Backward pass and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()
        optimizer.zero_grad()

        if (epoch + 1) > cos_epoch:
            schedule.step()

        total_loss += loss.item()
        tbar.set_description(f"Epoch {epoch + 1} Loss: {total_loss / (idx + 1)}")

    # Calculate average training loss
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation loop
    tbar = tqdm(val_loader)
    model.eval()
    val_preds = []
    val_loss = 0
    for idx, batch in enumerate(tbar):
        sequence = batch['sequence'].cuda()
        labels = batch['labels'].cuda()

        with torch.no_grad():
            output = model(sequence)
            labels = labels.view_as(output)
            #output = output.squeeze() ## Double check this 
            loss = criterion(output, labels)
            loss = loss.mean()

        val_loss += loss.item()
        val_preds.append([labels.cpu().numpy(), output.cpu().numpy()])

    val_loss = val_loss / len(val_loader)
    val_losses.append(val_loss)
    print(f"Epoch {epoch + 1} - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Save latest checkpoint
    scheduler_to_save = schedule if (epoch + 1) > cos_epoch else None
    save_checkpoint(epoch, model, optimizer, scheduler_to_save, 
                   avg_train_loss, val_loss, train_losses, val_losses, 
                   best_loss, checkpoint_dir, 'latest_checkpoint.pt')

    # Save the best model
    if val_loss < best_loss:
        best_loss = val_loss
        best_preds = val_preds
        
        # Save best weights 
        torch.save(model.state_dict(), 
                   os.path.join(os.environ['RNETEB_PATH'], 
                               'results/rnet_eb_weights/RibonanzaNet-EB_000_log_kds.pt'))
        
        # Save best checkpoint (full)
        save_checkpoint(epoch, model, optimizer, scheduler_to_save, 
                       avg_train_loss, val_loss, train_losses, val_losses, 
                       best_loss, checkpoint_dir, 'RNET_EB_000_best_checkpoint.pt')
        print(f"✓ New best model saved! Val Loss: {val_loss:.4f}")

    # Save periodic checkpoints every 10 epochs
    if (epoch + 1) % 10 == 0:
        save_checkpoint(epoch, model, optimizer, scheduler_to_save, 
                       avg_train_loss, val_loss, train_losses, val_losses, 
                       best_loss, checkpoint_dir, f'RNET_EB_000_checkpoint_epoch_{epoch+1}.pt')

    # Plot and save loss curves
    plot_loss_curve(train_losses, val_losses, figure_dir)

# Plot final summary
plot_final_summary(train_losses, val_losses, figure_dir)

print(f"\n{'='*50}")
print(f"Training Complete!")
print(f"Best Validation Loss: {best_loss:.4f}")
print(f"Checkpoints saved to: {checkpoint_dir}")
print(f"Loss curves saved to: {figure_dir}")
print(f"{'='*50}")

  0%|          | 0/1258 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/users/gtully/Ranger-Deep-Learning-Optimizer//ranger/ranger.py:138: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value = 1) (Triggered internally at /pytorch/torch/csrc/utils/python_arg_parser.cpp:1661.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
  0%|          | 0/629 [00:00<?, ?

Epoch 1 - Train Loss: 2.3541, Val Loss: 1.8239
✓ New best model saved! Val Loss: 1.8239


  0%|          | 0/1258 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  0%|          | 0/629 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 629/629 [00:17<00:00, 35.24it/s]


Epoch 2 - Train Loss: 0.9229, Val Loss: 0.8937
✓ New best model saved! Val Loss: 0.8937


  0%|          | 0/1258 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  0%|          | 0/629 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 629/629 [00:17<00:00, 35.26it/s]


...

  0%|          | 0/1258 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  0%|          | 0/629 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 629/629 [00:17<00:00, 35.30it/s] | 926/1258 [02:23<00:52,  6.38it/s] 


Epoch 7 - Train Loss: 0.4463, Val Loss: 0.6776
✓ New best model saved! Val Loss: 0.6776


  0%|          | 0/1258 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  0%|          | 0/629 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 629/629 [00:17<00:00, 35.23it/s]


Epoch 8 - Train Loss: 0.4102, Val Loss: 0.6400


  0%|          | 0/629 [00:00<?, ?it/s]/home/users/gtully/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Epoch 13 Loss: 0.29967843714155684:  15%|█▌        | 192/1258 [00:29<02:48,  6.32it/s]]